In [1]:
import pandas as pd
import ml_utils
import torch
import torch.nn.functional as F
from torch import nn
import torch.optim as optim
from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'torch'

In [ ]:
df = pd.read_csv("../output/FINAL/dataset.pca.60.csv")
df = df.sample(frac=1, random_state=1)

In [ ]:
mm = ml_utils.build_minmax_scaler(df)

In [ ]:
xtrain = mm.transform(df.query('dataset=="Train"').drop(columns=['uuid', 'cancer_type', 'dataset']))
ytrain = df.query('dataset=="Train"')['cancer_type']
ytrain_one_hot = pd.get_dummies(ytrain)

In [ ]:
xvalidation = mm.transform(df.query('dataset=="Validation"').drop(columns=['uuid', 'cancer_type', 'dataset']))
yvalidation = df.query('dataset=="Validation"')['cancer_type']
yvalidation_one_hot = pd.get_dummies(yvalidation)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
for depth in [5, 10]:
    for ne in [100, 200, 300, 400, 500]:
        model = RandomForestClassifier(max_depth=depth, n_estimators=ne)
        model.fit(xtrain, ytrain)
        print(depth, "-", ne, "--", model.score(xtrain, ytrain), model.score(xvalidation, yvalidation))

In [ ]:
yvalidation

## Neural Networks

In [ ]:
class DNN(nn.Module):
    
    def __init__(self):
        super(DNN, self).__init__()
        self.layer1 = nn.Linear(60,20)
        self.layer2 = nn.Linear(20,20)
        self.layer3 = nn.Linear(20,14)
        
    def forward(self, xin):
        x=xin
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        x = F.relu(x)
        x = self.layer3(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
dnn = DNN()
loss_fn = nn.CrossEntropyLoss()

In [ ]:
n_epochs = 100
batch_size = 64
learning_rate = 0.000001
optimizer = optim.Adam(dnn.parameters(), lr=learning_rate)

train_losses = []
validation_losses = []

for i_epoch in tqdm(range(n_epochs)):
    for i in tqdm(range(len(xtrain))):
        optimizer.zero_grad()
        xbatch = torch.from_numpy(xtrain[i:i+batch_size]).float()
        ybatch = torch.from_numpy(ytrain_one_hot[i:i+batch_size].values).float()
        ypred = dnn(xbatch)
        loss = loss_fn(ypred, ybatch)
        loss.backward()
        optimizer.step()
    ypred = dnn(torch.from_numpy(xtrain).float() )
    train_loss = loss_fn(ypred, torch.from_numpy(ytrain_one_hot.astype(float).values)).detach().numpy()
    train_losses.append(train_loss)
    ypred = dnn(torch.from_numpy(xvalidation).float() )
    val_loss = loss_fn(ypred, torch.from_numpy(yvalidation_one_hot.astype(float).values)).detach().numpy()
    validation_losses.append(val_loss)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses)
plt.plot(validation_losses, "--")

In [ ]:
predclasses = dnn(torch.from_numpy(xtrain).float()).argmax(dim=1)

In [ ]:
predictions = pd.Series(ytrain_one_hot.columns[predclasses])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(predictions, ytrain)

In [ ]:
dftrain=df.query('dataset=="Train"')
dftrain

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(
    data=dftrain,
    x='pca_1',
    y='pca_2',
    hue='cancer_type'
)